### Import Libraries

In [30]:
import os
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly as py
import plotly.express as px
import plotly.io as pio
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

### Import Useful Functions

In [31]:
parent_directory = os.path.join(os.environ["BLACK_BOX"])
validations_path = os.path.join(parent_directory, "experiments/results/validation_checkpoints")
sys.path.append(parent_directory)

from experiments.utils.validation_utils import extract_results_csv

### Get Algorithm Validation Results Paths

In [63]:
# validation algorithm resultant folders located in ./experiments/results/validation_checkpoints folder
folder_name = "beyesian_search_PPOTrainer_highway_environment:highway-environment-v0_2022-09-12_07-04-47d194k0gb_Agent2_bo_Chkpt294"
title_name = "BO Simulation of PPO Agent-2"

### Read Verification Results

In [ ]:
# read results.csv file inside given validation folder
df = extract_results_csv(os.path.join(validations_path, folder_name, "results.csv"))
df

### Print General Statistics

In [ ]:
n_samples = df.shape[0]
print("Number of Samples :\t", df.shape)

n_collisions = df["collision"].value_counts()[True]
print("Number of Collisions :\t", n_collisions)

### Extract Useful Columns

In [66]:
ego_v1 = df["config/ego_v1"].to_numpy()
front_v1 = df["config/front_v1"].to_numpy()
front_v2 = df["config/front_v2"].to_numpy()
delta_dist = df["config/delta_dist"].to_numpy()
min_ttc = df["episode_min_ttc"].to_numpy()

In [67]:
ego_v1_str = "ego speed-1 (m/s)"
front_v1_str = "mio speed-1 (m/s)"
front_v2_str = "mio speed-2 (m/s)"
delta_dist_str = "mio distance (m)"
min_ttc_str = "TTC (s)"

In [ ]:
configs_df = df[["config/ego_v1", "config/front_v1", "config/front_v2", "config/delta_dist", "episode_min_ttc"]]
configs_df = configs_df.rename(columns={
                                        "config/ego_v1": ego_v1_str,
                                        "config/front_v1": front_v1_str,
                                        "config/front_v2": front_v2_str,
                                        "config/delta_dist": delta_dist_str,
                                        "episode_min_ttc": min_ttc_str})
configs_df

### Remove Unnecessary (High TTC) Samples

In [ ]:
configs_df_risky = configs_df[configs_df[min_ttc_str] < 2.0]
configs_df_risky

In [ ]:
fig = px.scatter_3d(configs_df_risky,
                    x=ego_v1_str, y=front_v1_str, z=front_v2_str,
                    color=min_ttc_str,
                    size_max=1)
fig.show()

In [ ]:
fig = px.scatter_matrix(configs_df_risky,
                       dimensions=[ego_v1_str,
                                   front_v1_str,
                                   front_v2_str,
                                   delta_dist_str],
                       color=min_ttc_str,
                       color_continuous_scale="Rainbow", # Portland
                       opacity=1.0,
                       #title=title_name
                       )
fig.update_layout(
    font=dict(
        family="Times New Roman",
        size=18,
        color="black"
    )
)
# fig.update_traces(diagonal_visible=False)
# fig.update_layout({
#     "plot_bgcolor": "rgba(0, 0, 0, 0)",
#     "paper_bgcolor": "rgba(0, 0, 0, 0)",
# })
fig.show()

In [72]:
pio.write_image(fig, title_name + ".svg", width=1011, height=625)

In [ ]:
fig = ff.create_scatterplotmatrix(configs_df_risky,
                                  diag="box",
                                  index=min_ttc_str,
                                  colormap=px.colors.sequential.Viridis,
                                  height=500, width=800)
py.offline.iplot(fig, filename=title_name)

In [ ]:
px.colors.named_colorscales()